In [1]:
## Import required Python modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy, scipy.stats
import io
import base64
#from IPython.core.display import display
from IPython.display import display, HTML, Image
from urllib.request import urlopen

try:
    import astropy as apy
    import astropy.table
    _apy = True
    #print('Loaded astropy')
except:
    _apy = False
    #print('Could not load astropy')

## Customising the font size of figures
plt.rcParams.update({'font.size': 14})

## Customising the look of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))
## This custom file is adapted from https://github.com/lmarti/jupyter_custom/blob/master/custom.include
HTML('custom.css')
#HTML(urlopen('https://raw.githubusercontent.com/bretonr/intro_data_science/master/custom.css').read().decode('utf-8'))

In [2]:
## Custom imports
from scipy.stats import binom, poisson, chi2, norm, uniform
from scipy.optimize import curve_fit
from math import ceil, pi
from numpy import exp
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle, Rectangle
from matplotlib.colors import makeMappingArray

In [2]:
HTML('''
<script>
    function toggleCodeCells() {
      var codeCells = document.querySelectorAll('.jp-CodeCell');

      codeCells.forEach(function(cell) {
        var inputArea = cell.querySelector('.jp-InputArea');
        if (inputArea) {
          var currentDisplay = inputArea.style.display || getComputedStyle(inputArea).display;
          inputArea.style.display = currentDisplay === 'none' ? '' : 'none';
        }
      });
    }
</script>

<!-- Add a button to toggle visibility of input code cells -->
<button onclick="toggleCodeCells()">Toggle Code Cells</button>
''')

<div class="container-fluid">
    <div class="row">
        <div class="col-md-8" align="center">
            <h1>PHYS 10791: Introduction to Data Science</h1>
            <!--<h3>2019-2020 Academic Year</h3><br>-->
        </div>
        <div class="col-md-3">
            <img align='center' style="border-width:0" src="images/UoM_logo.png"/>
        </div>
    </div>
</div>

<div class="container-fluid">
    <div class="row">
        <div class="col-md-2" align="right">
            <b>Course instructors:&nbsp;&nbsp;</b>
        </div>
        <div class="col-md-9" align="left">
            <a href="http://www.renebreton.org">Prof. Rene Breton</a> - Twitter <a href="https://twitter.com/BretonRene">@BretonRene</a><br>
            <a href="http://www.hep.manchester.ac.uk/u/gersabec">Dr. Marco Gersabeck</a> - Twitter <a href="https://twitter.com/MarcoGersabeck">@MarcoGersabeck</a>
        </div>
    </div>
</div>

# Chapter 8 - Problem Sheet

### Problem 1: Type I and II errors

Identify which statements are correct.

- Type I error is the rate of acceptance of the hypothesis in a hypothesis test.
- Type I error is the rate of rejection of the hypothesis in a hypothesis test.
- Type I error is the rate of acceptance of the alternative hypothesis in a hypothesis test.
- Type I error is the rate of rejection of the alternative hypothesis in a hypothesis test.
- Type II error is the rate of acceptance of the hypothesis in a hypothesis test.
- Type II error is the rate of rejection of the hypothesis in a hypothesis test.
- Type II error is the rate of acceptance of the alternative hypothesis in a hypothesis test.
- Type II error is the rate of rejection of the alternative hypothesis in a hypothesis test.


### Solution to Problem 1

The correct statements are
- Type I error is the rate of rejection of the hypothesis in a hypothesis test.
- Type II error is the rate of acceptance of the alternative hypothesis in a hypothesis test.

### Problem 2: The choice of significance and power

#### Problem 2.1

Describe in your words what are the relevant things to consider when choosing the acceptance point of a hypothesis test, which defines significance and power.

#### Problem 2.2

In a medical diagnostic test that aims to identify a disease the quantities discussed are often: 
- the sensitivity, i.e. the rate at which true positives are not overlooked, and
- the specificity, i.e. the rate of candidates without a disease that are correctly identified as healthy.

Relate these to Type I and Type II errors and to significance and power.

#### Problem 2.3

A medical diagnostic test has a rate of Type I errors of $20\%$ and a rate of Type II errors of $0.01\%$. The test is carried out on 100,000 candidates. It is expected that 1 in 1,000 people carry the disease. Based on these numbers calculate
- the expected number of infected candidates,
- the expected number of candidates returning a postive test,
- the number of infected candidates not identified as such, and
- the fraction of postive tests that were returned by healthy candidates.

Based on the last number, discuss the usefulness of this test and what could be done to address this.

### Solution to Problem 2

#### Solution 2.3
- $1/1000 \times 100000 = 100$
- $100\times(1-0.2)+(100000-100)\times 0.01\% = 80 + 100 = 180$
- $100\times0.2 = 20$
- $100/180=56\%$

The question of the usefulness depends very much on the consequences of the test. If a positive test would have as a consequence an invasive treatment, then this should clearly not end up being applied to a majority of healthy individuals. If on the other hand, this is mostly to assess the prevalence of the disease in a population, which could even be corrected for the known rate of false positives, the test would be fine.

If the test was carried out twice on those positively identified, the picture changes dramatically:
- of the 80 true positive cases, 64 would still be identified as such a second time, while
- none of the false positive cases would be expected to return a second positive test ($100\times 0.01\% \ll 1$).

In this case one might even prefer a test with a greater rate of false positives if this would allow to reduce the number of false negatives.

### Problem 3: Hypothesis tests with Poisson and Gauss

The last lecture video discussed an example in which a Poisson distribution was approximated by a Gaussian distribution. This problem aims to illustrate this further. In a counting experiment, assume that the hypothesis is that the expected count rate is 30. Make a table for counts 0 to 50 with the following columns (if you're not using a computer and calculate the numbers one-by-one, you may start at a count of 15; note also that one of the parts of the Poisson probability formula does not depend on the count):
- The count (a running number from 0 to 50)
- The Poisson probability for this count
- The cumulative Poisson probability for counts from 0 to this value
- The signed number of standard deviations corresponding to the deviation of this count from the mean when approximating the Poisson distribution by a Gaussian normal distribution.
- The fractional integral of the Gaussian normal distribution up to the number of standard deviations calculated in the previous column
- The ratio of the cumulative sum of Poisson probabilities to the fraction integral of the normal distribution, i.e. of the values in the third and fifth column.

In [18]:
import numpy as np
import scipy, scipy.stats
from math import ceil, pi
from scipy.stats import poisson, norm, binom

mean = 30
ks = np.array(range(0,51))

rvp = poisson(mean)       # initialise poisson distribution
p_probs = rvp.pmf(ks)      # calculate poisson probabilities for all values of x and return list
p_ints = np.cumsum(p_probs, dtype=float) # calculate cumulative sum of probabilities

g_sigmas = (ks-mean)/mean**0.5
g_cumprob = norm.cdf(g_sigmas)

ratios = p_ints / g_cumprob

np.set_printoptions(precision=5,suppress=True)
for a,b,c,d,e,f in zip(ks,p_probs,p_ints,g_sigmas,g_cumprob,ratios):
    print(a,b,c,d,e,f)

0 9.357622968840175e-14 9.357622968840175e-14 -5.477225575051661 2.160231528913744e-08 4.331768536655691e-06
1 2.807286890652049e-12 2.900863120340451e-12 -5.294651389216606 5.96218342720841e-08 4.86543756286056e-05
2 4.210930335978075e-11 4.50101664801212e-11 -5.11207720338155 1.5931777314234595e-07 0.0002825181747921237
3 4.21093033597808e-10 4.661032000779292e-10 -4.929503017546495 4.1219531191084066e-07 0.0011307823903119716
4 3.1581977519835553e-09 3.6243009520614845e-09 -4.74692883171144 1.0326428980631015e-06 0.003509733092494493
5 1.894918651190139e-08 2.2573487463962875e-08 -4.564354645876384 2.5051659781952214e-06 0.009010775198306552
6 9.474593255950688e-08 1.1731942002346976e-07 -4.381780460041329 5.885669548807499e-06 0.019933062678865476
7 4.0605399668360107e-07 5.233734167070708e-07 -4.199206274206274 1.3392611712705536e-05 0.0390792645926222
8 1.522702487563499e-06 2.04607590427057e-06 -4.016632088371218 2.9517890247939978e-05 0.06931646832088087
9 5.075674958544996e-06

<div class="well" align="center">
    <div class="container-fluid">
        <div class="row">
            <div class="col-md-3" align="center">
                <img align="center" alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" width="60%">
            </div>
            <div class="col-md-8">
            This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>).
            </div>
        </div>
    </div>
    <br>
    <br>
    <i>Note: The content of this Jupyter Notebook is provided for educational purposes only.</i>
</div>